## Imports

In [1]:
import numpy as np 

## Reading Databases

In [2]:
class Vertex():
    def __init__(self, name):
        self.name = name
        self.edges = []
        self.weights = []
        self.pheromone = [] 
    
    def add(self, other, weight):
        self.edges.append(other)
        self.weights.append(weight)
        self.pheromone.append(10)  # Inicialize pheromone equals 10 to all edges

In [3]:
class Graph():
    def __init__(self):
        self.vertices = dict()
    
    def build_graph(self, file):
        with open(file) as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]
            for line in lines:
                line = line.split('\t')
                line = list(map(float, line))
                if line[0] not in self.vertices.keys():
                    self.vertices[line[0]] = Vertex(line[0])
                self.vertices[line[0]].add(line[1], line[2])
    
    def update_pheromone(self, origin, destine, cost):
            index = self.vertices[origin].edges.index(destine)
            self.vertices[origin].pheromone[index] += cost
    
    def evaporate_pheromone(self, evaporation_rate):
        for vertex in self.vertices.keys():
            #self.vertices[vertex].pheromone -= np.array(evaporation_rate)
            self.vertices[vertex].pheromone *= np.array(1-evaporation_rate)
            self.vertices[vertex].pheromone = np.clip(self.vertices[vertex].pheromone, a_min=0, a_max=None)
    
    def show_pheromone(self):
        for vertex in self.vertices.keys():
            print(self.vertices[vertex].pheromone)
    
    def reset_pheromone(self):
        for vertex in self.vertices.keys():
            self.vertices[vertex].pheromone = [10 for _ in self.vertices[vertex].pheromone]

In [4]:
file = 'graph1.txt'
G1 = Graph()
G1.build_graph(file)

## ACO 

In [5]:
class Ant():
    def __init__(self, destine):
        self.path = [1]
        self.destine = destine
    
    def show(self):
        for e in self.path[:-1]:
            print(e, end=', ')
        print(self.path[-1])
    
    def step(self, graph, alfa=1, beta=2):
        vertex = self.path[-1]
        edges = graph.vertices[vertex].edges
        pheromone = graph.vertices[vertex].pheromone
        weights = graph.vertices[vertex].weights
        index = [i for (i,e) in enumerate(edges) if e not in self.path]
        if index == []:
            
            return None, 0
        #edges = [e for (i,e) in enumerate(edges) if i in index]
        pheromone = [p for (i,p) in enumerate(pheromone) if i in index]
        v_weights = [w for (i,w) in enumerate(weights) if i in index]
        #print(edges)
        prob_num = np.array(pheromone)**alfa * np.array(v_weights)**beta
        prob = prob_num/np.sum(prob_num)
        chosen = np.random.choice(index, p=prob)
        destination = edges[chosen]
        cost = weights[chosen]
        self.path.append(destination)
        return destination, cost
        
    def walk(self, graph, alfa=1, beta=2):
        total_cost = 0
        destination = 1
        while(destination != self.destine and destination != None):
            destination, cost = self.step(graph, alfa, beta)
            # print(destination, len(self.path))
            ##Test
            ##if len(self.path) <= 99:
            ##     if destination == 100:
            ##            self.path.pop()
            ##            destination = 0
            ##            continue
            ## 
            total_cost += cost

        #self.show()
        if destination == None:
            print("DEAD END ")
            self.path=[1]  # Clear ant path before return 
            return 0, None
        #print("Total lenght = ", len(self.path))
        #print("Total weight = ", total_cost)
        
        # Reset ant path
        path = self.path 
        self.path = [1]  # Clear ant path before return 
        
        return total_cost, path


In [6]:
class ACO():
    def __init__(self, graph=None, number_of_ants=20, max_iterations=100, evaporation_rate=0.05, alfa=1, beta=2):
        self.number_of_ants = number_of_ants
        self.graph = graph
        self.ants = [Ant(len(self.graph.vertices)) for _ in range(self.number_of_ants)]
        self.max_iterations = max_iterations
        self.evaporation_rate = evaporation_rate
        self.current_iteration = 0
        self.best = (0, None)
        self.alfa = alfa
        self.beta = beta
        
    def iteration(self):
        self.current_iteration += 1
        
        #self.graph.show_pheromone()
        #Evaporate pheromone
        self.graph.evaporate_pheromone(self.evaporation_rate)
        
        fitness = []
        for ant in self.ants:
            
            cost, path = ant.walk(self.graph, self.alfa, self.beta)
            fitness.append(cost)
            if path == None:
                continue
            
            #Update best 
            if cost > self.best[0]:
                self.best = (cost, path)
        
            #Update pheromone
            for i, vertex in enumerate(path[:-1]):
                origin = vertex
                destine = path[i+1]
                self.graph.update_pheromone(origin, destine, cost/100)
        
        #print("Best so far: ", self.best[0])
        print("Best so far: {},  Average fitness: {}".format(self.best[0], np.mean(fitness)))
        
        # Return iteration fitness (mean and best) 
        return np.mean(fitness), self.best[0]
    
    def run(self, number_of_ants=20, max_iterations=100, evaporation_rate=0.05, alfa=1, beta=2):
        self.number_of_ants = number_of_ants
        self.max_iterations = max_iterations
        self.evaporation_rate = evaporation_rate
        self.alfa = alfa
        self.beta = beta
        print("-----------------")
        print("Parameters: ")
        print("N of ants: ", self.number_of_ants)
        print("Max iterations: ", self.max_iterations)
        print("Evaporation rate: ", self.evaporation_rate)
        print("Alfa: ", self.alfa)
        print("Beta: ", self.beta)
        print("-----------------", '\n')
        
        
        best_list = []
        mean_list = []
        for i in range(self.max_iterations):
            mean, best = self.iteration()
            mean_list.append(mean)
            best_list.append(best)
        return mean_list, best_list
    
    def reset(self):
        self.current_iteration = 0
        self.best = (0, None)
        self.graph.reset_pheromone()

In [7]:
fitness = [1,3, 4,5, 6]
x = np.mean(fitness)
x

3.8

## Problem 1

In [8]:
sol1 = ACO(G1)

In [9]:
number_of_ants = 20
max_iterations = 150
evaporation_rate = 0.05
sol1.run(number_of_ants, max_iterations, evaporation_rate)

-----------------
Parameters: 
N of ants:  20
Max iterations:  150
Evaporation rate:  0.05
Alfa:  1
Beta:  2
----------------- 

Best so far: 757.0,  Average fitness: 446.25
Best so far: 757.0,  Average fitness: 396.85
Best so far: 757.0,  Average fitness: 402.55
Best so far: 757.0,  Average fitness: 346.65
Best so far: 769.0,  Average fitness: 370.6
Best so far: 797.0,  Average fitness: 374.25
Best so far: 797.0,  Average fitness: 279.4
Best so far: 797.0,  Average fitness: 426.65
DEAD END 
Best so far: 825.0,  Average fitness: 408.7
Best so far: 849.0,  Average fitness: 347.8
Best so far: 849.0,  Average fitness: 294.45
Best so far: 849.0,  Average fitness: 404.65
Best so far: 849.0,  Average fitness: 365.4
Best so far: 849.0,  Average fitness: 345.05
Best so far: 849.0,  Average fitness: 394.9
Best so far: 849.0,  Average fitness: 336.85
Best so far: 849.0,  Average fitness: 277.65
Best so far: 849.0,  Average fitness: 375.95
Best so far: 849.0,  Average fitness: 397.7
Best so far: 

([446.25,
  396.85,
  402.55,
  346.65,
  370.6,
  374.25,
  279.4,
  426.65,
  408.7,
  347.8,
  294.45,
  404.65,
  365.4,
  345.05,
  394.9,
  336.85,
  277.65,
  375.95,
  397.7,
  294.35,
  320.55,
  292.3,
  413.2,
  381.8,
  356.95,
  349.1,
  343.0,
  336.25,
  263.15,
  270.2,
  338.95,
  424.65,
  306.3,
  400.05,
  323.25,
  337.1,
  339.8,
  267.1,
  358.75,
  389.2,
  319.35,
  283.9,
  352.4,
  284.95,
  406.45,
  355.35,
  265.55,
  334.4,
  251.35,
  285.15,
  338.9,
  358.0,
  304.4,
  280.25,
  299.25,
  329.7,
  351.75,
  293.05,
  333.1,
  326.5,
  317.25,
  332.8,
  287.45,
  302.6,
  378.85,
  297.2,
  334.7,
  345.3,
  344.95,
  245.55,
  350.4,
  298.05,
  307.4,
  367.35,
  343.4,
  331.0,
  298.65,
  336.2,
  330.2,
  311.05,
  299.15,
  228.4,
  354.8,
  326.9,
  288.0,
  326.7,
  304.05,
  371.8,
  317.15,
  274.35,
  334.5,
  349.65,
  303.2,
  316.45,
  364.05,
  275.0,
  320.9,
  281.75,
  364.1,
  298.95,
  292.7,
  240.9,
  262.85,
  338.45,
  288.3,
  

## Problem 2

In [10]:
file = 'graph2.txt'
G2 = Graph()
G2.build_graph(file)

In [11]:
number_of_ants = 20
max_iterations = 150
evaporation_rate = 0.01
alfa = 1
beta = 2
sol2 = ACO(G2, number_of_ants, max_iterations, evaporation_rate, alfa, beta)

In [12]:
sol2.reset()
sol2.run()

-----------------
Parameters: 
N of ants:  20
Max iterations:  100
Evaporation rate:  0.05
Alfa:  1
Beta:  2
----------------- 

DEAD END 
Best so far: 126.0,  Average fitness: 62.65
Best so far: 126.0,  Average fitness: 64.3
Best so far: 126.0,  Average fitness: 57.5
DEAD END 
Best so far: 129.0,  Average fitness: 58.05
Best so far: 134.0,  Average fitness: 67.65
Best so far: 134.0,  Average fitness: 61.55
DEAD END 
Best so far: 136.0,  Average fitness: 53.25
DEAD END 
Best so far: 136.0,  Average fitness: 58.9
Best so far: 136.0,  Average fitness: 66.4
DEAD END 
Best so far: 136.0,  Average fitness: 63.7
Best so far: 136.0,  Average fitness: 56.5
DEAD END 
Best so far: 136.0,  Average fitness: 49.1
Best so far: 136.0,  Average fitness: 62.35
Best so far: 152.0,  Average fitness: 69.1
Best so far: 152.0,  Average fitness: 65.95
Best so far: 152.0,  Average fitness: 58.45
DEAD END 
Best so far: 152.0,  Average fitness: 66.7
Best so far: 152.0,  Average fitness: 74.6
Best so far: 156.0,

([62.65,
  64.3,
  57.5,
  58.05,
  67.65,
  61.55,
  53.25,
  58.9,
  66.4,
  63.7,
  56.5,
  49.1,
  62.35,
  69.1,
  65.95,
  58.45,
  66.7,
  74.6,
  68.7,
  64.8,
  55.9,
  73.8,
  52.1,
  69.15,
  55.3,
  66.6,
  50.3,
  76.1,
  64.75,
  60.65,
  65.05,
  61.0,
  56.05,
  69.8,
  62.2,
  65.55,
  66.15,
  69.6,
  75.65,
  62.25,
  66.25,
  72.6,
  72.55,
  75.55,
  72.8,
  86.9,
  66.9,
  75.8,
  68.95,
  71.2,
  70.6,
  63.05,
  73.65,
  79.9,
  63.8,
  79.4,
  80.65,
  58.45,
  66.55,
  62.5,
  81.75,
  73.55,
  48.6,
  68.75,
  78.5,
  62.6,
  77.9,
  79.7,
  72.65,
  76.7,
  70.2,
  77.4,
  71.6,
  83.9,
  73.8,
  84.6,
  93.0,
  78.2,
  70.5,
  75.4,
  61.35,
  87.3,
  72.1,
  85.3,
  82.35,
  72.1,
  79.05,
  73.85,
  71.05,
  74.6,
  84.85,
  79.25,
  82.0,
  84.45,
  71.35,
  63.8,
  78.55,
  77.45,
  78.2,
  60.65],
 [126.0,
  126.0,
  126.0,
  129.0,
  134.0,
  134.0,
  136.0,
  136.0,
  136.0,
  136.0,
  136.0,
  136.0,
  136.0,
  152.0,
  152.0,
  152.0,
  152.0,
  15